In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Set up the WebDriver
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# First part: Searching for restaurants on Google Maps

# Replace with the URL of the webpage containing the <select> element
url = 'https://www.google.com/maps'
driver.get(url)
time.sleep(1)

# Variables for search criteria
# Prompt user for input
city = input("Enter the city: ")
landmark = input("Enter the landmark: ")
cuisine = input("Enter the cuisine (e.g., Japanese): ")
specialty = input("Enter the specialty (e.g., Chicken Karaage): ")
rating = float(input("Enter the minimum rating (0.0-5.0): "))
restrictions = input("Enter dietary restrictions: ")
features = input("Enter features: ")
service_options = input("Enter service options (Delivery, Takeout, Dine-in): ")

# Constructing search query
search_query = f"{cuisine} restaurant near {landmark}, {city} with {specialty}, {restrictions}, {features}, {service_options}"

# Typing search query into the search bar
search_box = driver.find_element(By.XPATH, "//input[@id='searchboxinput']")
search_box.send_keys(search_query)
time.sleep(0.5)

# Clicking the search button
driver.find_element(By.XPATH, '//*[@id="searchbox-searchbutton"]').click()

time.sleep(5)

# Applying filters (if needed)
if rating >= 2:
    driver.find_element(By.XPATH, '//*[@id="assistive-chips"]/div/div/div/div[1]/div/div/div/div/div[2]/div[2]/div[1]/button').click()
    driver.find_element(By.XPATH, f'//*[@id="action-menu"]/div[{int(rating * 2 - 3)}]').click()

# Second part: Processing individual restaurant details

# Collecting restaurant links
resto_links = []
resto_html = driver.find_elements(By.CLASS_NAME, "Nv2PK.THOPZb.CpccDe")

for restaurant in resto_html:
    try:
        link_element = restaurant.find_element(By.TAG_NAME, "a")
        href = link_element.get_attribute('href')
        if href:
            resto_links.append(href)
    except Exception as e:
        print(f"Error collecting link: {str(e)}")

# Loop through each restaurant link
for link in resto_links:
    driver.get(link)
    
    # Allow time for the page to load
    driver.implicitly_wait(10)
    
    # Find and print restaurant details
    try:
        name_element = driver.find_element(By.CLASS_NAME, "DUwDvf.lfPIob")
        name = name_element.text if name_element else "N/A"
        
        rating_element = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]')
        rating = rating_element.text if rating_element else "N/A"
        
        address_element = driver.find_element(By.CLASS_NAME, "rogA2c")
        address = address_element.text if address_element else "N/A"
        
        try:
            open_element = driver.find_element(By.XPATH, '//*[@class="o0Svhf"]')
            open = open_element.text if open_element else "N/A"
        except:
            open = "N/A"
        
        try:
            phone_element = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[9]/div[7]/button/div/div[2]')
            phone = phone_element.text if phone_element else "N/A"
        except:
            phone = "N/A"
        
        print(f"Restaurant: {name}")
        print(f"Rating: {rating}")
        print(f"Address: {address}")
        print(f"Open: {open}")
        print(f"Phone Number: {phone}")
        print("------------------------------------")
    
    except Exception as e:
        print(f"Error: {str(e)}")

# Close the WebDriver
driver.quit()
